# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import requests
import math
import zipfile
import os
import time

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

### Daniel Edit - I Download the newest list for free from Kaggle. I Traced the Download file to the real URL by inspecting the page and using requests to download the zip & zipfile library to unzip

In [2]:
def download_file(url):
    """
    This Function Downloads a file from the given URL stored in file and saves it as 'S&P.zip' in the current working directory.
    Returns the local file path.
    """
    local_filename = 'S&P.zip'
    # Use the requests library to download the file at the given URL
    with requests.get(url, stream=True) as r:
        # Raise an error if the request returns a non-200 status code
        r.raise_for_status()
        # Open the local file for writing in binary mode
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                # Write the current chunk to the local file
                f.write(chunk)
    # Add the current working directory to the filename
    local_filename = os.getcwd() + '\\' + local_filename
    print("your file is in: ",local_filename)
    f.close()
    return local_filename

def extract_from_zip(zip_file,inside_file):
    """
    Extracts all files from the given zip file that contain the specified string in their name.
    Extracts to the current working directory.
    """
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extract all files that contain the specified string in their name
        zip_ref.extractall(path = './', members = [m for m in zip_ref.namelist() if inside_file in m])


# Open the file 'DOWNLOAD_URL.txt' for reading ! u need to go to kaggle and start the download and cancel, then inspect the download link. this will be fixed later
with open ('DOWNLOAD_URL.txt','r') as u:
    # Read the contents of the file and store it as the URL
    URL=u.read()

file_name=download_file(URL)
extract_from_zip(file_name,'sp500_companies.csv')

your file is in:  C:\Users\Daniel\PycharmProjects\AlgoTrading\S&P.zip


Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks=pd.read_csv('sp500_companies.csv')
stocks = stocks.iloc[:,1] #we have all the data we need in the csv, but we want to get the most accureate data from IEX cloud
stocks

0       AAPL
1       MSFT
2       GOOG
3      GOOGL
4       AMZN
       ...  
488      NWL
489      ALK
490     NCLH
491      LNC
492     LUMN
Name: Symbol, Length: 493, dtype: object



## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from just_secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

### Daniel Note-The Old sandbox is deprecated. There is a brand new system. Therefore I'm using brand new documentation and real data instead of syntactic like the course does

In [5]:
symbol='MMM'
api_url=f'https://api.iex.cloud/v1/data/core/quote/BRK.B?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data[0]['week52High'])
# the data returned for the get command is in the form of a dictionary inside a list- [dict]
# therefor we point to the first object in the list, and then the key of the wanted value

362.1


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price=data[0]['latestPrice']
market_cap=data[0]['marketCap']
print (f'{symbol} market cap is {round(market_cap/1000000000000,3)} in Trillion dollars')

MMM market cap is 0.701 in Trillion dollars


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns= ['Symbol','Stock Price','Market Capitalization', 'Number of Shares to Buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Symbol,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
    ) ,
    ignore_index=True
)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_13268\597336784.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Symbol,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,317.64,700647201669,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop. this will take 350 seconds because each command is around 1 second! we will not use that and use batch call.

In [9]:
final_dataframe=pd.DataFrame(columns=my_columns)
dash='-'
start_time = time.time()
max_lop_time=0
for stock in stocks:
    s_lop_time=time.time()
    print(stock)
    if(dash in stock):
        #IEX symbols has a dot instead of dash '-' in the names of the stocks which are class A,B etc. there for we need to replace it.
        stock=stock.replace(dash,".")
    api_url=f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    new_row=pd.Series({'Symbol':stock,'Stock Price':data[0]['latestPrice'],'Market Capitalization':round(data[0]['marketCap']/1000000,3),'Number of Shares to Buy':'N/A'})
    final_dataframe=pd.concat([final_dataframe,new_row.to_frame().T], ignore_index=True)
    lop_time=time.time()-s_lop_time
    if (max_lop_time<=lop_time):
        max_lop_time=lop_time
        print("long! ",max_lop_time)
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time)
print("Longest lop:", max_lop_time)

AAPL
long!  0.7126150131225586
MSFT
GOOG
GOOGL
long!  0.7136135101318359
AMZN
BRK-B
JNJ


KeyboardInterrupt: 

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Daniel Note - The new IEX can take around 1000 request.

In [ ]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [ ]:
symbols=list(chunks(stocks['Symbol'],500))
symbols_strings=[]
for i in range(0,len(symbols)):
     symbols_strings.append(','.join(symbols[i]))
final_dataframe=pd.DataFrame(columns=my_columns)
start_time = time.time()
i=0
symbols_strings = [i.replace("-",".") for i in symbols_strings]         #IEX symbols has a dot instead of dash '-' in the names of the stocks which are class A,B etc. there for we need to replace it.
print(symbols_strings)
for s in symbols_strings:
    batch_url=f'https://api.iex.cloud/v1/data/core/quote/{s}?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_url).json()
    for symbol in s.split(','):
        print(symbol)
        print(data[i]['latestPrice'])
        print(data[i]['symbol'])
        print(data[i]['marketCap'])
        new_row=pd.Series({'Symbol':symbol,'Stock Price':data[i]['latestPrice'],'Market Capitalization':round(data[i]['marketCap']/1000000,3),'Number of Shares to Buy':'N/A'})
        final_dataframe=pd.concat([final_dataframe,new_row.to_frame().T], ignore_index=True)
        print(new_row)
        i+=1
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time)
final_dataframe

 ## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

### Daniel- Explanation - lets say we want to buy from each company equally regarding the amount of $

In [ ]:
while True:
    try:
        protfolio_size=input('Enter protfolio size')
        val=float(protfolio_size)
        print(val)
    except:
        print('Please enter an integer')
        continue
    else:
        break

In [ ]:
#A function that calculates the number of shares to buy based on the stock price
def calculate_shares(row):
    price = row['Stock Price']
    shares =math.floor(position_size/price)
    return shares

position_size=val/len(final_dataframe.index)
# Use the .apply() method to apply the calculate_shares function to each row of the DataFrame
#final_dataframe['Number of Shares to Buy'] = final_dataframe.apply(calculate_shares, axis=1)
final_dataframe['Total'] = final_dataframe['Stock Price'] * final_dataframe['Number of Shares to Buy']
total_spending = final_dataframe['Total'].sum()
total_spending = "{:.2f}".format(total_spending)
total_shares = final_dataframe['Number of Shares to Buy'].sum()
final_dataframe.loc['Total spending'] = ['Total', 0, 0 ,total_shares, total_spending]
print("Total spending: ", total_spending)
final_dataframe


## Formatting Our Excel Output - Not Used


### writing in Excel seems less exciting to me so I wanted to try to send those command into Interactive brokers sandbox program. the Interactive brokers software must be open with client ID 100 and ticked V at the most top setting under settings ->>API

In [ ]:
import time
from ibapi.client import *
from ibapi.wrapper import *
import threading

class TestApp(EClient, EWrapper): # intilaize EClient and Ewrapper
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: OrderId):
        self.nextOrderId = orderId
        print(f'Next valid order ID is {orderId}')

def run_loop():
    app.run()

app = TestApp()
app.connect("127.0.0.1", 7497, 100)

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()


time.sleep(1)# wait for the app to start

apple_contract=Contract()
apple_contract.symbol='AAPL'
apple_contract.secType='STK'
apple_contract.exchange='ISLAND'
apple_contract.currency='USD'


order=Order()
order.action="BUY"
order.orderType="MKT"
order.totalQuantity=1

app.placeOrder(app.nextOrderId,apple_contract,order)

time.sleep(10)# wait for the order to be placed
app.disconnect()
